In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#extracting data 
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",
    loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents

In [ ]:
# extracted_data=load_pdf_file(data="test_data/")
#extracted data

In [ ]:
extracted_data = load_pdf_file(
    data="C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\Data"
)
# print(extracted_data)


In [ ]:
print(len(extracted_data))


In [ ]:
#splitting data : chunks 
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print(f"Number of chunks", len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
# download embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [ ]:
import sys
print(sys.executable)

In [ ]:
embeddings = download_hugging_face_embeddings()


In [ ]:
query_result= embeddings.embed_query('HELLO world')
print("length",  len(query_result))
#query_result()

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=r"C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\.env")

In [ ]:
PINE_API_KEY=os.environ.get("PINECONE_api_key")
GEMINI_API_KEY=os.environ.get("Gemini_api_key")

In [ ]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec
# import os




# pc= Pinecone(api_key=PINE_API_KEY)
# index_name="therapybot"
# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [ ]:
#setting as environment var
import os
os.environ["PINE_API_KEY"]= PINE_API_KEY
os.environ["GOOGLE_API_KEY"]= GEMINI_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore
#embed chunks and upsert the embebded vectors to pinecone db ie index
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [ ]:
from langchain_pinecone import PineconeVectorStore
#embed chunks and upsert the embebded vectors to pinecone db ie index
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3}) 

In [ ]:
retrieved_docs=retriever.invoke("How to prevent suicide?")

In [ ]:
retrieved_docs

In [ ]:
#llm model is initialised
# from langchain_openai import OpenAI
# llm=OpenAI(temperature=0.4, max_tokens=500)

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.4,
    max_tokens=200  # or specify a limit if needed
    google_api_key=GEMINI_API_KEY  # Explicitly pass the API key

)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a highly experienced and qualified mental health counsellor for high school students. "
    "Use the retrieved context and the user’s prompt to answer, give advice, and suggest things to ease the problem of the user. "
    "Use an empathetic and warm tone to not excite any reaction but to support the user. "
    "Take a deep breath and process all the context, then give an answer which best just advises and suggests rather than straight up solving the problem. "
    "Ask questions and ask for more context if required. "
    "You are the single thing supporting and comforting the user in any given situation, so tread lightly so as to not get too personal yet act as a warm counsellor to take care of the mental health of the user. "
    "If you don’t have straight answers or if the answers are too complex, start by recommending to seek warmth and comfort from adults, teachers, counsellors, or booking an appointment with the student therapist. "
    "Give answers in simple wording, short and friendly manner. "
    "Use a suggestive style and avoid making it sound compulsory or like an order. "
    "Take time to give answers and don’t make up information entirely. Be kind and warm. "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
question_answer_chain= create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"I feel very anxious about my exams next week."})
print(response["answer"])